## **In This Notebook, I have built a Resume Parser Application using Llama2 model and langchain framework**

### Apart from Langchain, we also need to Install Auto GPTQ as we would be using Quantized version of Llama2 Model

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq openai==0.27.4 --progress-bar off
!pip install -Uqqq watermark==2.3.1 --progress-bar off
!pip install -Uqqq chromadb==0.4.5 --progress-bar off
!pip install -Uqqq tiktoken==0.3.3 --progress-bar off
!pip install -Uqqq youtube-transcript-api==0.5.0 --progress-bar off
!pip install -Uqqq pytube==12.1.3 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1  --progress-bar off
!pip install -qqq xformers==0.0.20  --progress-bar off
!pip install -Uqqq unstructured[local-inference]==0.5.12 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pubsublite 1.8.2 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.4.0 which is incompatible.
jupyterlab-lsp 4.2.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.


In [ ]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [ ]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [ ]:
import os
import textwrap
import torch
import chromadb
import langchain
import openai
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI, HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from auto_gptq import AutoGPTQForCausalLM
from transformers import AutoTokenizer, pipeline, logging, TextStreamer
from langchain.document_loaders.image import UnstructuredImageLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [ ]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

### Load Resume, Split into chunks of 2K characters, convert to embeddings using Instruct Embeddings and store in chroma DB

In [ ]:
pdf_loader = UnstructuredPDFLoader("/content/input/CV1_.pdf")

In [ ]:
pdf_pages = pdf_loader.load_and_split()

In [ ]:
len(pdf_pages)

2

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=512)
texts = text_splitter.split_documents(pdf_pages)
len(texts)

4

In [ ]:
model_name = "hkunlp/instructor-large"

hf_embeddings = HuggingFaceInstructEmbeddings(
    model_name = model_name) ## , model_kwargs = {'device': 'cuda'}

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
db = Chroma.from_documents(texts, hf_embeddings)

### Initialize the Quantized Llama2 Model from Huggingface, and warp it by Langchain's HuggingFace Pipeline.

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "gptq_model-4bit-128g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device='cuda:0',
        use_triton=use_triton,
        quantize_config=None)

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt = True, skip_special_tokens = True)
text_pipeline = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, streamer = streamer)
llm = HuggingFacePipeline(pipeline = text_pipeline)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

### Prompt function

In [ ]:
def generate_prompt(prompt, sys_prompt):
    return f"[INST] <<SYS>> {sys_prompt} <</SYS>> {prompt} [/INST]"

In [ ]:
sys_prompt = "Use following piece of context to answer the question in less than 20 words"
template = generate_prompt(
    """
    {context}

    Question : {question}
    """
    , sys_prompt)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

### Initialize Langchain's Q&A Chain

In [ ]:
chain_type_kwargs = {"prompt": prompt}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs,
)

### Have a chat with Resume

In [ ]:
result = qa_chain("what projects candidate worked on ?")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 Sure! Here's the answer to the question in less than 20 words:

Candidate worked on uplift modeling, model uncertainty estimation, chat intent classification, and machine verse projects.


In [ ]:
result = qa_chain("where did candidate study?")

 Sure! Here's the answer to your question in less than 20 words:

Candidate studied at IIT Kharagpur.


In [ ]:
result = qa_chain("what skillset candidate has?")

 Based on the provided context, the candidate has the following skillset:

* Experience in deriving insights and intelligence from Credit Bureau, Card transaction, and closed loop Data.
* Strong knowledge of multiple ML domains like Time Series, Model Interpretability, NLP, Computer Vision.
* Leadership experience as a Manager, leading a team of 3 Data Scientists.
* Proficient in using various data science tools such as Transformer, Shapelet, CNN, LSTM, GNN, GBM, and SHAP, Lime, Integrated Gradient.
* Familiarity with programming languages such as Python, Pandas, NumPy, and PySpark, Hive.
* Strong understanding of Credit Risk, Credit Bureaus, Closed Loop Transaction, Spend Analytics, Loan Portfolio, and Credit Line.


In [ ]:
result = qa_chain("what's the contact detail of candidate?")

 Based on the given context, the contact details of the candidate are:

dd1996@gmail.com


In [ ]:
result = qa_chain("how many years of work experience candidate has?")

 Based on the given context, the candidate has 4 years of work experience.


In [ ]:
result = qa_chain("In which companies with what jobtitle did candidate work?")

 Sure! Here is the answer to your question in less than 20 words:

Candidate worked at American Express in a leadership role as a Data Scientist, and also worked at other companies with job titles such as Data Scientist and Manager.
